In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
!pwd

/Users/isaac/Documents/GitHub/SteamStore-ANALYSE/data


In [2]:
df = pd.read_json("valide_data.jsonl",lines=True)
df

,app_id,nom,image,type,description_courte,description_detaillee,est_gratuit,date_de_sortie,developpeurs,editeurs,...,genres,categories,tags_utilisateurs,plateformes,configuration_requise,support_manette,langues,evaluations,donnees_commerciales,contenu_et_fonctionnalites
0,20820,Shatter,https://shared.akamai.steamstatic.com/store_it...,jeu,Shatter is a retro-inspired brick-breaking gam...,Shatter is a retro-inspired brick-breaking gam...,False,"15 Mar, 2010",[Sidhe],[PikPok],...,"[Indie, Action]","[Single-player, Steam Cloud, Remote Play Toget...","[Great Soundtrack, Indie, Arcade, Action, Loca...","[windows, mac, linux]",{'pc': {'minimum': {'os': 'Windows XP SP3 or h...,None,"{'support_audio_complet': [], 'support_partiel...","{'metacritic_score': 84, 'recommandations_tota...","{'prix_initial': 159, 'prix_final': 159, 'pour...",{'nombre_succes': 30}
1,27050,Fatale,https://shared.akamai.steamstatic.com/store_it...,jeu,"""Salomé, Salomé, dance for me. I pray thee dan...","""Salomé, Salomé, dance for me. I pray thee dan...",False,"4 Nov, 2009",[Tale of Tales],[Tale of Tales],...,"[Casual, Indie, Adventure]","[Single-player, Family Sharing]","[Indie, Adventure, Casual, Short, Psychologica...","[windows, mac]",{'pc': {'minimum': {'os': 'Windows XP/Vista/7'...,None,"{'support_audio_complet': [], 'support_partiel...","{'metacritic_score': None, 'recommandations_to...","{'prix_initial': 599, 'prix_final': 599, 'pour...",{'nombre_succes': 0}
2,1002,Rag Doll Kung Fu,https://shared.akamai.steamstatic.com/store_it...,jeu,A piece of Steam history - THE FIRST EVER NON ...,A piece of Steam history - THE FIRST EVER NON ...,False,"12 Oct, 2005",[Mark Healey],[Mark Healey],...,[Indie],"[Single-player, Family Sharing, Multi-player]","[2D Fighter, Martial Arts, Intentionally Awkwa...",[windows],{},None,"{'support_audio_complet': [], 'support_partiel...","{'metacritic_score': 69, 'recommandations_tota...","{'prix_initial': 79, 'prix_final': 79, 'pource...",{'nombre_succes': 0}
3,15400,Harvest: Massive Encounter,https://shared.akamai.steamstatic.com/store_it...,jeu,Harvest: Massive Encounter is an award-winning...,Harvest: Massive Encounter is an award-winning...,False,"23 Jan, 2009",[Oxeye Game Studio],[Oxeye Game Studio],...,"[Indie, Strategy]","[Single-player, Family Sharing, Steam Achievem...","[Strategy, Indie, Tower Defense, Sci-fi, Singl...","[windows, mac, linux]",{'pc': {'minimum': {'os': 'Windows 2000/XP/Vis...,None,"{'support_audio_complet': [], 'support_partiel...","{'metacritic_score': None, 'recommandations_to...","{'prix_initial': 659, 'prix_final': 659, 'pour...",{'nombre_succes': 25}
4,20828,Shatter - Original Soundtrack,https://shared.akamai.steamstatic.com/store_it...,jeu,The award-winning Shatter Official Videogame S...,The award-winning Shatter Official Videogame S...,False,"30 Jul, 2012",[Sidhe],[PikPok],...,"[Indie, Action]",[Single-player],"[Action, Indie, Soundtrack]","[windows, mac, linux]",{},None,"{'support_audio_complet': [], 'support_partiel...","{'metacritic_score': None, 'recommandations_to...","{'prix_initial': 159, 'prix_final': 159, 'pour...",{'nombre_succes': 0}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81185,341610,Grass Simulator - Soundtrack,https://shared.akamai.steamstatic.com/store_it...,dlc,"Grass Simulator - Soundtrack, includes eight o...",Grass Simulator – Soundtrack! – Now available ...,False,"1 Apr, 2015","[Conor Strejcek, Daniel Davies]",[Daniel Davies],...,"[Simulation, Indie, Casual]","[Family Sharing, Downloadable Content, Single-...","[Indie, Casual, Simulation]","[windows, mac, linux]","{'pc': {'minimum': {'os': 'XP/Vista/7/8', 'sto...",None,"{'support_audio_complet': [], 'support_partiel...","{'metacritic_score': None, 'recommandations_to...","{'prix_initial': 299, 'prix_final': 299, 'pour...",{'nombre_succes': 0}
81186,301248,Fantasy Grounds - C&C: U1 Shadows of Halfling ...,https://shared.akamai.steamstatic.com/store_it...,dlc,U1: Shadows of the H

In [3]:
# 1. Fonction pour compter 
def get_count(val):
    if val is None or (isinstance(val, float) and val is None): return 0
    if isinstance(val, list): return len(val)
    return 0

# feature enginnering 
def filter_langs(lang_list_game, allowed_langs):
    if not lang_list_game:
        return []
    
    kept = [lang for lang in lang_list_game if lang in allowed_langs]
    
    # Si on a supprimé des langues → ajouter "Autre"
    if len(kept) < len(lang_list_game):
        kept.append("Autres")
    
    return kept

In [4]:
df = df[df["type"]=="jeu"]
df = df[df["est_gratuit"]==False]

df["nombre_succes"] = df["contenu_et_fonctionnalites"].apply(lambda x: x["nombre_succes"])
df["prix"] = df["donnees_commerciales"].apply(lambda x: x["prix_initial"])
df["tmp"]= df["evaluations"].apply(lambda x: x["avis_utilisateurs"])
df["review"] = df["tmp"].apply(lambda x: x["total"])
df["pourcentage_positif"] = df["tmp"].apply(lambda x: x["pourcentage_positif"])
df["metacritic_score"] = df["evaluations"].apply(lambda x: x["metacritic_score"])
df["recommandations_total"] = df["evaluations"].apply(lambda x: x["recommandations_total"])

df["langues_partiel"] = df["langues"].apply(lambda x: x['support_partiel'])
df["langues_complet"] = df["langues"].apply(lambda x: x['support_audio_complet'])

df["N_langues_partiel"] = df["langues_partiel"].apply(get_count)
df["N_langues_complet"] = df["langues_complet"].apply(get_count)

df["support_manette"] = df["support_manette"].eq("full")
df["date_de_sortie"] = pd.to_datetime(df["date_de_sortie"], errors="coerce")
df["annee"] = df["date_de_sortie"].dt.year
df["mois"]  = df["date_de_sortie"].dt.month
df["succes"] = df["succes"] = (df["review"] > 500) & (df["pourcentage_positif"] > 85)

In [5]:
n_games = len(df)

langs = (
    df["langues_partiel"]
    .explode()
    .dropna()
)
langs = langs[langs != ""]

lang_percent = (
    langs.value_counts()
    .div(n_games)
    .sort_values(ascending=False)
)

# Conversion en liste Python
lang_list2 = [(lang, round(p * 100, 2)) for lang, p in lang_percent.items() if p*100 > 1]

langs = (
    df["langues_complet"]
    .explode()
    .dropna()
)
langs = langs[langs != ""]

lang_percent = (
    langs.value_counts()
    .div(n_games)
    .sort_values(ascending=False)
)

# Conversion en liste Python
lang_list = [(lang, round(p * 100, 2)) for lang, p in lang_percent.items() if p*100 > 1]
allowed_langs = {lang for lang, _ in lang_list}

In [6]:
df["langues_complet"] = df["langues_complet"].apply(lambda x: filter_langs(x, allowed_langs))
df["langues_partiel"] = df["langues_partiel"].apply(lambda x: filter_langs(x, allowed_langs))

In [7]:
df_autre = df[df["langues_complet"].apply(lambda x: "Autres" in x)]
df_autre2 = df[df["langues_partiel"].apply(lambda x: "Autres" in x)]
print(len(df_autre),len(df_autre2))

583 964


In [ ]:
df = df[df['prix'] <= 100*100]

11393

In [15]:
df["prix"]

0        159
1        599
2         79
3        659
4        159
        ... 
81167      0
81172      0
81173      0
81181      0
81188     79
Name: prix, Length: 53616, dtype: int64

In [13]:
df2 = df.drop(columns=["type","est_gratuit","configuration_requise","evaluations","tmp","contenu_et_fonctionnalites","donnees_commerciales","franchise","date_de_sortie","langues"])
df2
train_df, test_df = train_test_split(
    df2,
    test_size=0.1, 
    random_state=42,
    shuffle=True
)

In [14]:
test_df.to_parquet("test.parquet")
train_df.to_parquet("train.parquet")